# Scraping Content from Coca Cola's Transparency Page
This is a simple script that takes a list of organization's from Coke's transparency page (www.coca-colacompany.com/transparency) and converts it to a CSV for easy parsing. 

The page articulates various research efforts and partnerships of the Coca Cola Company. 

Note that there are some places in the original data where contributions were made over the course of multiple years. In those cases, the total amount was divided by the number of years the contributions were made. 

In [38]:
from bs4 import BeautifulSoup
import requests, pandas as pd, re
from decimal import Decimal #The Decimal module was used to round currency to two decimals 
p = re.compile(ur'[\$\,\ ]') #Text for Regular expression to remove currency formatting. 
format = lambda x: Decimal(re.sub(p,'',x))

In [39]:
data=requests.get('http://www.coca-colacompany.com/transparency-search?noCache=true').text

In [40]:
soup=BeautifulSoup(data,'html.parser')
frame=pd.DataFrame(columns=['Title','Type','Organization','Amount','Date']) #Build the initial dataframe

In [41]:
for row in soup.findAll('tr'):
    datarow=[]
    for tabledata in row.findAll('td'):
        if(tabledata.find('span',{'class':'expandedTd'})):
            datarow.append(tabledata.find('span',{'class':'expandedTd'}).text.strip().encode('ascii','ignore'))
        else:
            datarow.append(tabledata.text.strip().encode('ascii','ignore'))
    if len(datarow)!=0:
        frame=frame.append(pd.DataFrame([datarow],columns=['Title','Type','Organization','Amount','Date']))
    
frame['Amount']=frame['Amount'].map(format)
fourchardates=(frame.loc[frame['Date'].str.len()==4])
longdates=(frame.loc[frame['Date'].str.len()>4])
expandeddates=pd.DataFrame(columns=['Title','Type','Organization','Amount','Date'])
for row in longdates.iterrows():
    for year in range(int(row[1][4].split('-')[0]),int(row[1][4].split('-')[1])+1):
    
        expandeddates=expandeddates.append((pd.DataFrame([[row[1][0],row[1][1],row[1][2],Decimal(row[1][3]/len(range(int(row[1][4].split('-')[0]),int(row[1][4].split('-')[1])+1))).quantize(Decimal('.01')),year]],columns=['Title','Type','Organization','Amount','Date'])))
alldates=fourchardates.append(expandeddates)

In [42]:
alldates.to_csv("Coca Cola Transparency Page.csv")